# Plotting
`scipp` offers a number of different ways to plot data from a `DataArray` or a `Dataset`.

In [ ]:
import numpy as np
import scipp as sc
from scipp import Dim

There are currently two different backends for plotting. The default uses the `plotly` package to render interactive graphs, well suited to `jupyter` notebooks, while the second employs the more classical and widespread `matplotlib` library.

Here we switch to the `matplotlib` backend, as `plotly` currently does not work for the documentation pages on Read the Docs.

In [ ]:
sc.plot_config.backend = "plotly"

## Plotting 1-D data

### 1-D line plot

Plotting is done using the `scipp.plot` function.
Generally the information in a dataset is sufficient to produce a useful plot out of the box.

For example, a simple line plot is produced as follows:

In [ ]:
d = sc.Dataset()
N = 50
d.coords[Dim.Tof] = sc.Variable([Dim.Tof], values=np.arange(N).astype(np.float64),
                                unit=sc.units.us)
d['Sample'] = sc.Variable([Dim.Tof], values=10.0*np.random.rand(N),
                          unit=sc.units.counts)
sc.plot(d)

### 1D line plot with error bars

Error bars are shown automatically if variances are present in the data:

In [ ]:
d['Sample'].variances = np.square(np.random.rand(N))
sc.plot(d)

Note that the length of the errors bars is the standard-deviation, i.e., the square root of the variances stored in the data.

### Multiple lines on the same axes

If a dataset contains more than one 1D variable with the same coordinates, they are plotted on the same axes:

In [ ]:
d['Background'] = sc.Variable([Dim.Tof], values=5.0*np.random.rand(N),
                              unit=sc.units.m)
sc.plot(d)

We can always plot just a single item of the dataset:

In [ ]:
sc.plot(d['Background'])

### Choosing the line colors

Line colors can be changed via the `color` keyword argument:

In [ ]:
sc.plot(d, color=['red', '#30D5F9'])

### Histograms
Histograms are automatically generated if the coordinate is bin edges:

In [ ]:
d['Histogram'] = sc.Variable([Dim.Tof], values=20.0*np.random.rand(N-1),
                             unit=sc.units.counts)
sc.plot(d['Histogram'])

and with error bars

In [ ]:
d['Histogram'].variances = 5.0*np.random.rand(N-1)
sc.plot(d['Histogram'])

The histogram color can be customized:

In [ ]:
sc.plot(d['Histogram'], color="#000000")

### Multiple datasets

`scipp.plot` also suports multiple 1-D datasets:

In [ ]:
other = sc.Dataset()
N = 60
other.coords[Dim.Tof] = sc.Variable([Dim.Tof],
                                    values=np.arange(N).astype(np.float64),
                                    unit=sc.units.us)
other['OtherSample'] = sc.Variable([Dim.Tof], values=10.0*np.random.rand(N),
                                   unit=sc.units.counts)
other['OtherNoise'] = sc.Variable([Dim.Tof], values=10.0*np.random.rand(N-1),
                                  variances=3.0*np.random.rand(N-1),
                                  unit=sc.units.counts)
sc.plot([d, other])

## Plotting 2-D data

### 2-D data as an image

2-D variables are plotted as images, with a colormap:

In [ ]:
N = 100
M = 50
xx = np.arange(N, dtype=np.float64)
yy = np.arange(M, dtype=np.float64)
x, y = np.meshgrid(xx, yy)
b = N/20.0
c = M/2.0
r = np.sqrt(((x-c)/b)**2 + ((y-c)/b)**2)
a = np.sin(r)
d1 = sc.Dataset()
d1.coords[Dim.X] = sc.Variable([Dim.X], values=xx, unit=sc.units.m)
d1.coords[Dim.Y] = sc.Variable([Dim.Y], values=yy, unit=sc.units.m)
d1['Signal'] = sc.Variable([Dim.Y, Dim.X], values=a, unit=sc.units.counts)
sc.plot(d1)

The dimension displayed along each axis of the image can be selected with the `axes` keyword argument which accepts a list of dimensions:

In [ ]:
sc.plot(d1, axes=[Dim.X, Dim.Y])

### 2-D data as filled contours

Instead of a classical image, we can also used filled contours to display the data:

In [ ]:
sc.plot(d1, contours=True)

### 2-D data with variances

If variances are present, they are not displayed by default, but they can be shown alongside the data values by using `show_variances=True`:

In [ ]:
d1['Signal'].variances = np.random.rand(M, N) * a * 0.1
sc.plot(d1, show_variances=True)

If interactive plotting is enabled in the `jupyter` notebook (either using the `plotly` backend or running `%matplotlib notebook` at the start of the notebook), zooming on either the values or the variances panel will also update the counterpart panel.

### Changing the colorscale

Changing the colorscale is handled via the `cb` keyword argument which is a dictionary holding different options. The type of colormap is defined by the `name` parameter:

In [ ]:
sc.plot(d1, cb={"name": "jet"})

A logarithmic colorscale is obtained by setting `log` to `True`:

In [ ]:
sc.plot(d1, cb={"name": "RdBu", "log": True})

Upper and lower limits on the colorscale can be placed using `min` and `max`:

In [ ]:
sc.plot(d1, cb={"min": 0, "max": 0.5})

And this can also be applied to the variances:

In [ ]:
sc.plot(d1, show_variances=True,
        cb={"min": 0, "max": 0.5, "min_var": -0.01, "max_var": 0.01})

### Collapsing dimensions

Sometimes it is useful to collapse one or more of the data's dimensions. This is done by specifying the dimension to be displayed along the x axis as a keyword argument. All other dimensions will be collapsed.

In [ ]:
N = 40
M = 5
d2 = sc.Dataset()
d2.coords[Dim.Tof] = sc.Variable([Dim.Tof],
                                 values=np.arange(N+1).astype(np.float64),
                                 unit=sc.units.us)
d2.coords[Dim.X] = sc.Variable([Dim.X], values=np.arange(M).astype(np.float64),
                               unit=sc.units.m)
d2['sample'] = sc.Variable([Dim.X, Dim.Tof], values=10.0*np.random.rand(M, N),
                           variances=np.random.rand(M, N))
sc.plot(d2)
sc.plot(d2, collapse=Dim.Tof)

## Plotting data with 3 and more dimensions

Data with 3 or more dimensions are currently represented by a 2-D image, accompanied by sliders to navigate the extra dimensions (one slider per dimension above 2). **Note that this is currently only supported by the plotly backend.**

In [ ]:
N = 50
M = 40
L = 30
K = 20
xx = np.arange(N, dtype=np.float64)
yy = np.arange(M, dtype=np.float64)
zz = np.arange(L, dtype=np.float64)
qq = np.arange(K, dtype=np.float64)
x, y, z, q = np.meshgrid(xx, yy, zz, qq, indexing='ij')
b = N/20.0
c = M/2.0
d = L/2.0
r = np.sqrt(((x-c)/b)**2 + ((y-c)/b)**2 + ((z-d)/b)**2  + ((q-d)/b)**2)
a = np.sin(r)
d3 = sc.Dataset()
d3.coords[Dim.X] = sc.Variable([Dim.X], values=xx)
d3.coords[Dim.Y] = sc.Variable([Dim.Y], values=yy)
d3.coords[Dim.Z] = sc.Variable([Dim.Z], values=zz)
d3.coords[Dim.Qx] = sc.Variable([Dim.Qx], values=qq)
d3['Some3Ddata'] = sc.Variable([Dim.X, Dim.Y, Dim.Z, Dim.Qx], values=a,
                               variances=np.random.rand(N, M, L, K) * a * 0.1)
sc.plot(d3, backend="plotly")

By default, the two innermost dimensions are used for the image, and the rest will be allocated to a slider. This can be changed by specifying the order of the axes

In [ ]:
sc.plot(d3, axes=[Dim.Z, Dim.Qx, Dim.Y, Dim.X], backend="plotly")

It is also possible to use a 3d projection:

In [ ]:
sc.plot(d3, backend="plotly", projection="3d")

And with variances:

In [ ]:
sc.plot(d3, backend="plotly", projection="3d", show_variances=True)

## Plotting datasets with mixed data shapes

If a dataset contains a mix of variables with different numbers of dimensions, a figure for each type is drawn:

In [ ]:
N = 60
M = 5
d4 = sc.Dataset()
d4.coords[Dim.Tof] = sc.Variable([Dim.Tof],
                                 values=np.arange(N).astype(np.float64),
                                 unit=sc.units.us)
d4['Sample1D'] = sc.Variable([Dim.Tof], values=10.0*np.random.rand(N),
                             unit=sc.units.counts)
d4['Noise1D'] = sc.Variable([Dim.Tof], values=10.0*np.random.rand(N-1),
                            variances=3.0*np.random.rand(N-1),
                            unit=sc.units.counts)
d4.coords[Dim.X] = sc.Variable([Dim.X], values=np.arange(M).astype(np.float64),
                               unit=sc.units.m)
d4['Image2D'] = sc.Variable([Dim.X, Dim.Tof], values=10.0*np.random.rand(M, N),
                            variances=np.random.rand(M, N))
sc.plot(d4)